## Agent Based Modeling (ABM): Forest Fire Model 3
-----------

This material is based on the previous version of CS108L, see https://cs4all.cs.unm.edu/cs108.html
- Credit to Prof. Melanie Moses, Dr. Irene Lee, Dr. Maureen Psaila-Dombrowski, Prof. Dave Ackley and collaborators (see website)

### Learning Goals

1. Continue learning how to use Mesa ABM 
    1. How to use ABM
        - Conduct experiments by changing values and variables (like how flammable a tree is)
        - Collect and analyze results
    2. Learn how to modify ABM
    3. Learn how to create ABM
        
        
2. Continue using Forest Fire Model to run multiple experiments with different parameters (density, forest size, ...)
    1. Agents are trees
    2. Trees are randomly scattered on a grid with a user-defined density
    3. Each step (iteration of the Runtime loop)
        - If a tree is on fire, all neighboring trees also catch fire

### Learning Goals

3. Continue learning how to put your forest fire experiment inside a loop over many experiments
    - That is, you will have one loop over many experiments (say 40)
    - And inside that loop, you will carry out one experiment
    - Compile average data on how much of the forest survives over all your experiments
   
4. New: Give trees a percent chance of catching on fire if a neighbor is burning
    - Currently all trees next to a burning tree catch fire with 100% probability

### First, we have to make sure that Mesa is installed, and then import our basic packages
- Note that Pandas and Seaborn or "built-in" packages, so we only have to import them, not install them

In [1]:
! pip install mesa==2.1.1


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [2]:
%matplotlib inline

import mesa

# Data visualization tools.
import seaborn as sns

# Data manipulation and analysis.
import pandas as pd

import matplotlib.pyplot as plt


### Now, we can begin creating our forest fire model 

### First, we define  the TreeAgent (inherits from `mesa.Agent`)


- Each Step (iteration of the Runtime loop)
   - If a tree is on fire, all neighboring trees also catch fire
   - Remember that the model step calls each agent step in random order


In [3]:
# Create ABM

# 1. Define the Agent Abstraction
class TreeAgent(mesa.Agent):

    def __init__(self, model, pos, probability_fire_spreads):
        """
        Create a new tree.
        
        Parameters:
            model: the agent based model, within which this agent is acting
            
            pos: The tree's coordinates on the grid. Used as the unique_id
        """
        
        super().__init__(pos, model)
        self.pos = pos
        self.unique_id = pos
        self.condition = "No Fire"
        self.probability_fire_spreads = probability_fire_spreads
        
    def step(self):
        """
        If the tree is on fire, spread it to `no fire` trees nearby.
        """
        if self.condition == "On Fire":
            neighbors = self.model.grid.get_neighbors(self.pos, moore=False)
            for neighbor in neighbors:
                if neighbor.condition == "No Fire":
                    if self.random.random() < self.probability_fire_spreads:
                        neighbor.condition = "On Fire"
            self.condition = "Burned Out"

### Next, we define our ForestFireModel 
1. Randomly place trees on a grid according to a `density`

   For each square on the grid
    - If a random number between 0 and 1 is below `density`
    - Then place a tree in that square

   Thus, each grid cell in our forest has a `density` percent chance that a tree will be placed on it at the start of the simulation.


2. Store the spatial locations of the agents for animations in `agent_locations`

### The `ForestFireModel` defines two more of our key abstractions:
1. This defines the environment where the agents (trees) exist
2. This also defines how to take a step (tick) forward in time


In [4]:
# 2. Define the Model (or environment) Abstraction for the Agents to live in
class ForestFireModel(mesa.Model):
    """
    Simple Forest Fire model.
    """

    def __init__(self, width, height, density, probability_fire_spreads):
        """
        Create a new forest fire model.

        Args:
            width, height: The size of the grid to model
            density: What fraction of grid cells have a tree in them.
        """
        # Set up model objects
        self.schedule = mesa.time.RandomActivation(self)
        self.grid = mesa.space.SingleGrid(width, height, torus=False)
        self.width = width
        self.height = height
        self.probability_fire_spreads = probability_fire_spreads
        
        # Save the spatial location of the agents in this list
        self.agent_locations = []
        
        # Store the number of trees with no fire, yes on fire, burned out 
        self.no_fire = []
        self.yes_fire = []
        self.burned_out = []
        self.percent_burned = []

        
        # Place a tree in each cell with Prob = density
        for x in range(self.width):
            for y in range(self.height):
                if self.random.random() < density:
                    # Create a tree
                    new_tree = TreeAgent(self, (x, y), probability_fire_spreads)
                    # Set all trees in the first column on fire.
                    if x == 0:
                        new_tree.condition = "On Fire"
                    self.grid[x][y] = new_tree
                    self.schedule.add(new_tree)
                    
        self.running = True

    def step(self):
        """
        Advance the model by one step.
        """
        self.schedule.step()
            
        # Store spatial locations, and whether a tree is on fire or not
        import numpy as np
        agent_counts = np.zeros((self.grid.width, self.grid.height))
        for cell_content, (x, y) in self.grid.coord_iter():
            if cell_content == None:
                # No tree here, label 0
                agent_counts[x][y] = 0
            else: 
                # Now, we know that this cell has a tree and that cell_content is a tree agent
                if cell_content.condition == "No Fire":
                    agent_counts[x][y] = 1
                elif cell_content.condition == "On Fire":
                    agent_counts[x][y] = 2
                elif cell_content.condition == "Burned Out":
                    agent_counts[x][y] = 3
                    
        self.agent_locations.append(agent_counts)
        
        # Count the number of trees with no fire, yes on fire, burned out 
        count_no_fire = 0
        count_yes_fire = 0
        count_burned_out = 0
        for tree in self.schedule.agents:
            if tree.condition == "No Fire":
                count_no_fire += 1
            elif tree.condition == "On Fire":
                count_yes_fire += 1    
            elif tree.condition == "Burned Out":
                count_burned_out += 1    
        ##
        self.no_fire.append(count_no_fire)
        self.yes_fire.append(count_yes_fire)
        self.burned_out.append(count_burned_out)
        
        # Store percent burned
        self.percent_burned.append( self.burned_out[-1] / (self.no_fire[0]+self.yes_fire[0]+self.burned_out[0]) )

### The below cell will generate animations.  
- Run this cell, to declare (define) this function
- This function is provided for you.  You only need to know how to use it. It generate animations of your agents as they move around on the grid.

In [5]:
# Run this cell
def plot_agents(model):
    import numpy as np
    import seaborn as sns
    import matplotlib.animation
    import matplotlib.pyplot as plt
    import matplotlib
    
    try:
      # if in a notebook, do inline
      get_ipython().run_line_magic('matplotlib', 'inline')
    except:
      pass

    plt.rcParams['figure.dpi'] = 72
    plt.rcParams["animation.html"] = "jshtml" # javascript html writer
    plt.ioff() # Turn interactive mode off
    plt.rcParams["figure.figsize"] = [7, 7]

    
    fig, ax = plt.subplots()
    global counter 
    counter = 0
    
    def animate(t):
        global counter
        agent_counts = model.agent_locations[t]
        
        cmap_dict = {0: '#FFFFFF', 1: '#00e400', 2: '#ff0000', 3: '#544c4a'}
        cmap = matplotlib.colors.ListedColormap([cmap_dict[i] for i in range(4)])
        
        # Plot using seaborn
        if counter == 0:
            g = sns.heatmap(agent_counts, vmin=0, vmax=3, cmap=cmap, cbar=True, square=True, ax=ax)
            c_bar = ax.collections[0].colorbar
            c_bar.set_ticks([0.5, 1.15, 1.9, 2.65])
            c_bar.set_ticklabels(['Empty', 'Tree', 'On Fire', 'Burned Out'])
        else:
            g = sns.heatmap(agent_counts, vmin=0, vmax=3, cmap=cmap, cbar=False, square=True, ax=ax)

        g.figure.set_size_inches(7, 7)
        g.set(title="Forest Fire Model")
        counter = counter+1

        
    anim = matplotlib.animation.FuncAnimation(fig, animate, frames=len(model.agent_locations), interval=200, repeat=False)
    return anim


### Task: Update the `ForestFireModel` to use the changes from the last lab
- That is, `ForestFireModel` should create and use the list `percent_burned`

### Task: Extend `ForestFireModel` so that trees catch fire with a probability

Currently, there is a 100% chance that a tree catches on fire, whenever a neighboring trees is on fire.

We will make this more realistic by assigning a probability here.

You will want to find this code

        if neighbor.condition == "No Fire":
            neighbor.condition = "On Fire"
        
And change it so that `neighbor.condition = "On Fire"` is executed (run) based on a percent.

Your new code will look something like

        ... update __init__ so that the user can specify a value for probability_fire_spread
        ... the ForestFireModel class should have a self.probability_fire_spreads value between 0 and 1.0
        ... and then this probability needs to get passed to the TreeAgent __init__ function
        ... and the self.probability_fire_spreads is assigned (set) inside __init__
        ...
        ...
        if neighbor.condition == "No Fire":
            if self.random.random() < self.probability_fire_spreads:
                ... set neighbor condition to On Fire



### Task: Set the `probability_fire_spread` to 0.8 and run the below code cell.

### Note that `ForestFireModel` now has the four parameters.  The new, final parameter should specify `probability_fire_spreads`

### Run the below cell for various densities and probabilities.  

In [6]:
fire_model = ForestFireModel(50, 50, 0.7, 0.75)
#                                          ^
#                                          |
#                                          |
#                                      this is the new value specifying probability_fire_spread


for i in range(50):
    fire_model.step()

anim = plot_agents(fire_model)
anim

### Task: Determine average behavior over many experiments

Next, create a new code cell below and put the above experiment inside a loop.  You can reuse code from your previous lab.

Your new code should have this structure.

       for 40 experiments
       
           final_percentage_burned = []
       
           for j in range(40):
                ... run one experiment with enough steps...
                ... final_percentage_burned.append(...) ...
                
The list `final_percentage_burned` should contain the percent of the forest burned down at the END
- That is, `final_percentage_burned[k]` should contain the percent of the forest burned down at the end of experiment `k`
- If you run 40 experiments, the `final_percentage_burned` will be a list with 40 entries

### Task continued:

Then, look at basic statistics to understand how the forest is affected by fire on average.  You can use pandas

        df = pd.DataFrame({'final_percentage_burned': final_percentage_burned})
        print("Averge percent of forest left: ", df['final_percentage_burned'].mean())
        ... print the same for df['final_percentage_burned'].median()
        ... print the same for df['final_percentage_burned'].min()
        ... print the same for df['final_percentage_burned'].max()
 

### Task: Copy and paste your above cell over 40 experiments to create new cells below
  - Copy and paste multiple times, so that you run multiple experiments for various densities and various probabilities for fire spreading. 
  - Make sure to comptue the mean, median, min, and max statistics for each experiment
      - Compute mean, median, min, and max in the same code cell in which you run your 40 experiments
      
  - Consider trying densities of 0.3, 0.5, 0.6, 0.8 (four sets of experiments), with a high probability that fire spreads
  
  - Then do another set of experiments for the probability fires spreads. 
      - Consider choosing one or two densities and then considering `probability_fire_spread` values of 0.25, 0.5, 0.75, 1.0.
      - This will generate four sets of experiment.
    
    
### Task: Write at least 600 words for your  results discussion below

### Begin experiments over density, for a probability of 0.9

In [7]:
final_percentage_burned = []

for i in range(40):
        
    fire_model = ForestFireModel(50, 50, 0.3, 0.9)

    for i in range(75):
        fire_model.step()
    
    final_percentage_burned.append(fire_model.percent_burned[-1])

df = pd.DataFrame({'final_percentage_burned': final_percentage_burned})
print("Averge percent of forest burned: ", df['final_percentage_burned'].mean())
print("Median percent of forest burned: ", df['final_percentage_burned'].median())
print("Max percent of forest burned: ", df['final_percentage_burned'].max())
print("Min percent of forest burned: ", df['final_percentage_burned'].min())

Averge percent of forest burned:  0.03222687949068677
Median percent of forest burned:  0.028453403960846453
Max percent of forest burned:  0.07095046854082998
Min percent of forest burned:  0.016602809706257982


In [8]:
final_percentage_burned = []

for i in range(40):
        
    fire_model = ForestFireModel(50, 50, 0.5, 0.9)

    for i in range(75):
        fire_model.step()
    
    final_percentage_burned.append(fire_model.percent_burned[-1])

df = pd.DataFrame({'final_percentage_burned': final_percentage_burned})
print("Averge percent of forest burned: ", df['final_percentage_burned'].mean())
print("Median percent of forest burned: ", df['final_percentage_burned'].median())
print("Max percent of forest burned: ", df['final_percentage_burned'].max())
print("Min percent of forest burned: ", df['final_percentage_burned'].min())

Averge percent of forest burned:  0.08349660482350289
Median percent of forest burned:  0.07380066896367783
Max percent of forest burned:  0.16233766233766234
Min percent of forest burned:  0.03423566878980892


In [9]:
final_percentage_burned = []

for i in range(40):
        
    fire_model = ForestFireModel(50, 50, 0.7, 0.9)

    for i in range(75):
        fire_model.step()
    
    final_percentage_burned.append(fire_model.percent_burned[-1])

df = pd.DataFrame({'final_percentage_burned': final_percentage_burned})
print("Averge percent of forest burned: ", df['final_percentage_burned'].mean())
print("Median percent of forest burned: ", df['final_percentage_burned'].median())
print("Max percent of forest burned: ", df['final_percentage_burned'].max())
print("Min percent of forest burned: ", df['final_percentage_burned'].min())

Averge percent of forest burned:  0.898909246106556
Median percent of forest burned:  0.918096009253904
Max percent of forest burned:  0.9618015963511972
Min percent of forest burned:  0.6390328151986183


====> NOTE that there is a big jump in burning when going from a density of 0.5 to 0.6
 THis is the kind of hting you want ot note in your writeup

In [10]:
final_percentage_burned = []

for i in range(40):
        
    fire_model = ForestFireModel(50, 50, 0.6, 0.9)

    for i in range(75):
        fire_model.step()
    
    final_percentage_burned.append(fire_model.percent_burned[-1])

df = pd.DataFrame({'final_percentage_burned': final_percentage_burned})
print("Averge percent of forest burned: ", df['final_percentage_burned'].mean())
print("Median percent of forest burned: ", df['final_percentage_burned'].median())
print("Max percent of forest burned: ", df['final_percentage_burned'].max())
print("Min percent of forest burned: ", df['final_percentage_burned'].min())

Averge percent of forest burned:  0.2504618037663295
Median percent of forest burned:  0.19208586563067948
Max percent of forest burned:  0.724025974025974
Min percent of forest burned:  0.06970509383378017


In [11]:
final_percentage_burned = []

for i in range(40):
        
    fire_model = ForestFireModel(50, 50, 0.8, 0.9)

    for i in range(75):
        fire_model.step()
    
    final_percentage_burned.append(fire_model.percent_burned[-1])

df = pd.DataFrame({'final_percentage_burned': final_percentage_burned})
print("Averge percent of forest burned: ", df['final_percentage_burned'].mean())
print("Median percent of forest burned: ", df['final_percentage_burned'].median())
print("Max percent of forest burned: ", df['final_percentage_burned'].max())
print("Min percent of forest burned: ", df['final_percentage_burned'].min())

Averge percent of forest burned:  0.9851502743094874
Median percent of forest burned:  0.9851170465108049
Max percent of forest burned:  0.9925410243659871
Min percent of forest burned:  0.9703628002043945


### Now run experiments over probability fire spreads for a fixed density

In [12]:
final_percentage_burned = []

for i in range(40):
        
    fire_model = ForestFireModel(50, 50, 0.7, 0.25)

    for i in range(75):
        fire_model.step()
    
    final_percentage_burned.append(fire_model.percent_burned[-1])

df = pd.DataFrame({'final_percentage_burned': final_percentage_burned})
print("Averge percent of forest burned: ", df['final_percentage_burned'].mean())
print("Median percent of forest burned: ", df['final_percentage_burned'].median())
print("Max percent of forest burned: ", df['final_percentage_burned'].max())
print("Min percent of forest burned: ", df['final_percentage_burned'].min())

Averge percent of forest burned:  0.025150125725703888
Median percent of forest burned:  0.024798187545896702
Max percent of forest burned:  0.0327774583093732
Min percent of forest burned:  0.018296169239565466


In [13]:
final_percentage_burned = []

for i in range(40):
        
    fire_model = ForestFireModel(50, 50, 0.7, 0.5)

    for i in range(75):
        fire_model.step()
    
    final_percentage_burned.append(fire_model.percent_burned[-1])

df = pd.DataFrame({'final_percentage_burned': final_percentage_burned})
print("Averge percent of forest burned: ", df['final_percentage_burned'].mean())
print("Median percent of forest burned: ", df['final_percentage_burned'].median())
print("Max percent of forest burned: ", df['final_percentage_burned'].max())
print("Min percent of forest burned: ", df['final_percentage_burned'].min())

Averge percent of forest burned:  0.04532317790441002
Median percent of forest burned:  0.04317937701396348
Max percent of forest burned:  0.07975460122699386
Min percent of forest burned:  0.022818026240730177


In [14]:
final_percentage_burned = []

for i in range(40):
        
    fire_model = ForestFireModel(50, 50, 0.7, 0.75)

    for i in range(75):
        fire_model.step()
    
    final_percentage_burned.append(fire_model.percent_burned[-1])

df = pd.DataFrame({'final_percentage_burned': final_percentage_burned})
print("Averge percent of forest burned: ", df['final_percentage_burned'].mean())
print("Median percent of forest burned: ", df['final_percentage_burned'].median())
print("Max percent of forest burned: ", df['final_percentage_burned'].max())
print("Min percent of forest burned: ", df['final_percentage_burned'].min())

Averge percent of forest burned:  0.28000919544449837
Median percent of forest burned:  0.2303151861332322
Max percent of forest burned:  0.7320924985899605
Min percent of forest burned:  0.0757396449704142


In [15]:
final_percentage_burned = []

for i in range(40):
        
    fire_model = ForestFireModel(50, 50, 0.7, 1.0)

    for i in range(75):
        fire_model.step()
    
    final_percentage_burned.append(fire_model.percent_burned[-1])

df = pd.DataFrame({'final_percentage_burned': final_percentage_burned})
print("Averge percent of forest burned: ", df['final_percentage_burned'].mean())
print("Median percent of forest burned: ", df['final_percentage_burned'].median())
print("Max percent of forest burned: ", df['final_percentage_burned'].max())
print("Min percent of forest burned: ", df['final_percentage_burned'].min())

Averge percent of forest burned:  0.9734716308209684
Median percent of forest burned:  0.9754033102870312
Max percent of forest burned:  0.9926925238898258
Min percent of forest burned:  0.9312754804892254


===> Note that the probability fire spreads has a big impact when going from 0.75 to 1.0
   This is the kind of thing that you want to note in your writeup

### Before writing your discussion
1. Read this article on forest thinning
https://www.abqjournal.com/lifestyle/fighting-fire-with-prevention-city-project-works-to-ensure-the-health-safety-of-the-bosque/article_097e666e-b3e6-11ee-be3a-37223009f7f7.html#5

2. Read this article on resident's reactions to the thinning
https://www.kunm.org/local-news/2024-03-01/albuquerque-official-responds-to-residents-concerns-about-the-bosque-thinning-project

3. This video on a production code for modeling forest fires 
https://www.youtube.com/watch?v=Xg6rciZtyhU

### In your discussion, consider the following.  

Think about sustainability and the natural and human worlds.  When do you think it makes sense to thin a forest to prevent fires?  What are the downsides?  What are the positives to thinning a forest?


### Enter a discussion of 600 words in below.  Have sections on the following four discussion points.

1. Evidence acquisition 
     - Describe the experiments you conducted. Give all the details, number of agents, steps, abstractions used, and so on. 
     - You are describing application of a quantitative model (the agent based model)

2. Evidence evaluation 
    - Discuss your plots and any other results.  What did you find? 
    - You are communicating and representing quantitative information
    
3. Conclusion
    - What can you conclude about your model
    - What is still unexplained? 
    
    
4. Conclusions and reflections about sustainability and the natural and human worlds
    

### In particular, here you want to focus on these questions

### Previous Questions: 

- What number of steps are needed to make sure the forest fire has stopped spreading?
    - How many steps do you need for quality evidence acquisition?

- How does density affect the forest fire model?  
    - Be specific, like, "A density of 0.5 represents a probability of 50% that a grid cell in our forest will have a tree in it at the start of the simulation.  With a density of 0.5, we see that the forest fire is likely to ..."

- Can this inform how we might manage a forest in real life?

- What is the effect of starting the fire in the middle?  Is the forest more likely to burn down, for a given density?

### New Question:
- What is the effect of varying the probability a neighboring tree catches on fire?  
- Discuss specifics here.  For instance, if a density of 0.8 and probability of fire spreading of 1.0 essentially guarantee that the forest will burn down --- Then, what must density change to, or the probability of fire spreading change to, in order to protect the forest from completely burning? 

### Homework Discussion Goes here

### Homework Submission Instructions

### The homework submission instructions are different this week. 

### Download your notebook as an ".ipynb" file, and upload the ipynb file as your assignment to Canvas.

### Do not upload a .py file